# On the Importance of Errors (oh, and Operator Overloading)

You may remember seeing the following code in the ‘Philosophy’ lectures:

In [5]:
def force(mass, acceleration):
    return mass * acceleration

I argued that one of the main drawbacks of this code it that "[it] implicitly assumes the user knows to pass in floats". 

And then I proposed two possible fixes to this issue, (1) add documentation explaining the codes purpose or (b) add in some error checking. To jog your memory, here's the code:

In [6]:
# DOCUMENTATION SOLUTION ... 
def force(mass, acceleration):
    """
    Calculates f=ma, returns force.
    We assume mass, acceleration are of type float.
    """
    return mass * acceleration

# ERROR CHECKING SOLUTION ...
def force(mass, acceleration):
    assert mass == type(float)         
    assert acceleration == type(float)  

    force = mass * acceleration
    return force

In those lectures past I proposed two possible fixes but I never actually explained what the problem was. That changes now. Here's the problem: 

In [7]:
print( "✓"  +  "✓")     # strings (rememeber strings support *any* unicode character)
print(  1   +   2)      # integers
print( [1]  +  [2])     # lists
print( (1,) +  (2,))    # tuples      

# Also possible:
# Class Object + Class Object (e.g Time(2,3) + Time(0,10) <-- see OOP lecture)

✓✓
3
[1, 2]
(1, 2)


The salient point is to notice that the "+" operator works not just on integers, but also tuples, lists, strings, class objects and so on. This is what we call operator overloading; it happens when a single operator (in this case "+") has several different meaning/uses, depending on context. For numbers, + means addition, for strings in means concatenation, and for other objects (i.e. custom classes) it could mean literally whatever we want it to mean.

In many ways, this is a great feature since it means the language ends up being a bit more concise and we have less symbols to remember. But you must be careful; There are bugs ahead!

Lets return to our function "force" shall we, lets call it with a bunch of objects (of different types) and see what happens!

In [8]:
force = lambda x, y: x * y  
# This Lambda function is equivilant to the "force" function above, but shorter. :)

print(force(10, 10))
print(force("2", 10))
print(force([1], 10 ))

100
2222222222
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


So in the three cases above we give the force function two integers, a string and an integer, and a list and an integer. **In each case the code "worked", and I'd argue that’s the problem!**

## Errors love you, love them back

> "Errors should never pass silently". ~ Zen of Python

As beginners you might hate it when Python throws error messages at you. You got something wrong and now you are being mocked for it. I’d counter that by saying this is very unhealthy attitude to have regarding error messages. Personally, I'd rather an error message over junk data any day of the week! 

Why? I'll give you two reasons...

Suppose you have medical data that you feed through a computer. It parses the data and the computer crashes on some error. What happens next? You reboot, fix, and re-run. Annoying sure, but no big deal.

Now imagine instead of getting an error Python just returns the wrong answer; think about the consequences of that just for a moment, wrong answers could easily have a disastrous impact on patient care.
    
Another reason you should prefer error messages instead of bad output is because an error in one place of code will probably cause an error much further on down the line (if you are lucky!). And the longer the error goes unoticed the harder it is to debug. I actually have example of exactly that happening in the ‘common error messages’ lecture. 

**Just to be clear, I’m not worried about feeding the force function dodgy input and the computer crashing. Rather, I’m concerned about those cases where the error may ‘sneak through’ unnoticed and we get a bad result.** And that could actually happen very very easily. In fact, let me show you just how easily we could get the wrong result. 

But instead of calling our function "force", lets change its name to  “Chemotherapy_radiation_dose”. The function is going to be exactly the same functionally, the name change is just for dramatic effect. I'm just gonna run with the medical disaster theme I’ve got going on here.

In [9]:
def chemotherapy_radiation_dose(patient_weight, cancer_stage):
    radiation_dose = patient_weight * cancer_stage
    return radiation_dose

print(chemotherapy_radiation_dose(65, 2))
print(chemotherapy_radiation_dose("65", 2))

130
6565


Okay, we have called the chemo function twice and we get the number 130 and the string 6565 returned to us. We shall use those values in a jiffy. Okay, now lets suppose that part of our program has a 'parse results' function. The task of this function is to get the results ready for printing. 

In [10]:
def parse_results(number):
    return "The correct doesage for the patient is" + number

parse_results(130)

TypeError: Can't convert 'int' object to str implicitly

So in the above code I called parse_results with the number 130 and the it returns an error. Now, since are expecting to be given numbers as input and we want to return a string it makes sense to use str() to convert the number. Afterall, a program that spins errors when we give it correct input is of no use at all.

So okay, we add str(number) to avoid spinning errors on valid inputs. Thats totally fine. BUT notice there is a nasty and insidious side-effect to doing this; from this point on "6535" is going to look like good data because all numbers are now strings. 

Just stop and think about that for a moment. 

From this point on the only clue you have that something went wrong is the sheer magnitude of the number, **ERROR MESSAGES CAN'T SAVE US NOW!**

Okay, let's further suppose that there is some other function that takes the string generated by 'parse results' and stores the numbers in log10, and it is this number the human operator of the 'big-cancer-zappy-machine' sees...

    log10(130)  = 2.11394335231
    log10(6565) = 3.81723473043
    
... and now the sheer magnitude of the fuck-up is masked. It is so easy to imagine some unaware lab technician not realising that there is a huge difference between 2.11 and 3.81. 

The result? A patient gets a radiation dose 50x more powerful than it should be!

## Therac-25, or why this shit matters...

And before you consider all this a bit far-fetched, it is worth considering that medical accidents involving software bugs have happened before. The Therac-25 was a radiation therapy machine that on least six separate occasions delivered a radiation dose 100x more powerful than they should have been; all due to a software bug. 

A commission looking into these incidents cited several safety breaches and poor practices that lead to this disaster. For our purposes, this paragraph stands out:

> "The system noticed that something was wrong and halted the X-ray beam, but merely displayed the word "MALFUNCTION" followed by a number from 1 to 64. The user manual did not explain or even address the error codes, so the operator pressed the P key to override the warning and proceed anyway."  ~ [Wikipedia article](https://en.wikipedia.org/wiki/Therac-25#Root_causes)

What can we learn from this? 

1. It might not be a good idea to simply report "MALFUNCTION 15" when you are killing people (readability counts).
2. If you must have a error message as abstruse as "MALFUNCTION 15" it might be a good idea to jot down "This means you are killing people" somewhere (documentation matters).  
3. Maybe make it harder to kill people than merely typing "P" on the keyboard? (serious errors should be given weight)

Basically the Therac-25 machine did raise errors but the mistake made was to not have a those errors mean anything to the end-user. Because the danger was not made explicit people literally died; When things go wrong it is crucial that error messages are not only raised but also respected.